In [242]:
import pandas as pd
import re
import numpy as np

In [243]:
df = pd.read_csv("day10.txt", header=None)
df.columns = ["coding"]
df

,coding
0,F-L----F7-7.F-7JJ.F--L-.F|F77.L|7.FLF-7.JJ--7F...
1,J|F7.LL|.F7.FL|J-77--JJ7L|-|-F-7L7-|LJ|F|JL|L-...
2,FF7J.7L|-7|FF7|L7|L|.LF7-JFLJJ.J7JJFJ-7.L-LLJJ...
3,FJ.LF-7-FJLJ-|7-7J777..|-FJ7.FJ.|.F|LFL-L7L|.....
4,|FF7LFL7LJJL|.|.||L77--|7F---L.F|77JFL7.F|7FL7...
...,...
135,|-|||JF-7-7||LJ7J-|7|..F7LFJ7.F7F|.F7.---7||J....
136,|.LFJ-|JLF7-7-||L|||J.FFJ7.|F-7J|7.LL-7|FJF-7F...
137,F-|.F7|.FJ7FL-|-7F||L-J|F7F-L7J-7--F7FFLJLFFL7...
138,.FJF|JFFJLL77||L-7LJ77F-||L-|J.||J.L7-7-L--77F...


In [244]:
x_dim = df.coding.str.len()[0]
y_dim = len(df)
x_dim, y_dim

(140, 140)

In [245]:
symbols = np.zeros((x_dim, y_dim), dtype="str")

In [246]:
starting_node = None

In [247]:
numbers = []
for y, row in enumerate(df.coding):
    for x, val in enumerate(row):
        symbols[y, x] = val
        if val == "S":
            starting_node = y, x
symbols

array([['F', '-', 'L', ..., '-', '7', '-'],
       ['J', '|', 'F', ..., '7', '.', '|'],
       ['F', 'F', '7', ..., '7', 'F', '-'],
       ...,
       ['F', '-', '|', ..., '|', 'J', '7'],
       ['.', 'F', 'J', ..., '7', '|', '|'],
       ['F', '-', 'J', ..., '.', '-', 'J']], dtype='<U1')

In [248]:
starting_node

(25, 108)

In [249]:
visited_nodes = {starting_node: 0}

In [250]:
from collections import deque
queue = deque()
queue.append(starting_node)

In [251]:
class SymbolDefinition:
    def __init__(self, symbol, new_locations_generator) -> None:
        self.symbol = symbol
        self.new_locations_generator = new_locations_generator

    def get_new_locations(self, location):
        y, x = location
        return self.new_locations_generator(y, x)

In [252]:
definitions = [
SymbolDefinition("-", lambda y, x: [(y, x-1), (y, x+1)]),
SymbolDefinition("|", lambda y, x: [(y+1, x), (y-1, x)]),
SymbolDefinition("L", lambda y, x: [(y, x+1), (y-1, x)]),
SymbolDefinition("J", lambda y, x: [(y, x-1), (y-1, x)]),
SymbolDefinition("7", lambda y, x: [(y, x-1), (y+1, x)]),
SymbolDefinition("F", lambda y, x: [(y, x+1), (y+1, x)])
]

In [253]:
symbol_def_map = {symboldef.symbol: symboldef for symboldef in definitions}

In [254]:
def process_locations(new_locations, prev_x, prev_y):
    current_steps = visited_nodes[(prev_y, prev_x)]
    for new_loc in new_locations:
        if new_loc not in visited_nodes:
            visited_nodes[new_loc] = current_steps + 1
            queue.append(new_loc)

while len(queue):
    current_search = queue.popleft()
    y, x = current_search
    #print(f"processing {current_search}")
    symbol = symbols[y, x]
    if symbol == "S":
        if symbols[y, x+1] == "-":
            visited_nodes[(y, x+1)] = 1
            queue.append((y, x+1))
        if symbols[y, x-1] == "-":
            visited_nodes[(y, x-1)] = 1
            queue.append((y, x-1))
        if symbols[y+1, x] == "|":
            visited_nodes[(y+1, x)] = 1
            queue.append((y+1, x))
        if symbols[y-1, x] == "|":
            visited_nodes[(y-1, x)] = 1
            queue.append((y-1, x))
        if symbols[y-1, x] == "J":
            visited_nodes[(y-1, x)] = 1
            queue.append((y-1, x))
        if symbols[y, x+1] == "J":
            visited_nodes[(y, x+1)] = 1
            queue.append((y, x+1))
        if symbols[y+1, x] == "L":
            visited_nodes[(y+1, x)] = 1
            queue.append((y+1, x))
        print(queue)
    else:
        symbol_def = symbol_def_map[symbol]
        new_locations = symbol_def.get_new_locations(current_search)
        process_locations(new_locations, x, y)   



deque([(24, 108), (26, 108)])


In [256]:
visited_nodes

{(25, 108): 0,
 (24, 108): 1,
 (26, 108): 1,
 (23, 108): 2,
 (26, 109): 2,
 (22, 108): 3,
 (25, 109): 3,
 (21, 108): 4,
 (24, 109): 4,
 (20, 108): 5,
 (23, 109): 5,
 (20, 107): 6,
 (23, 110): 6,
 (21, 107): 7,
 (22, 110): 7,
 (22, 107): 8,
 (22, 109): 8,
 (23, 107): 9,
 (21, 109): 9,
 (24, 107): 10,
 (20, 109): 10,
 (25, 107): 11,
 (19, 109): 11,
 (26, 107): 12,
 (19, 108): 12,
 (27, 107): 13,
 (19, 107): 13,
 (28, 107): 14,
 (19, 106): 14,
 (29, 107): 15,
 (18, 106): 15,
 (30, 107): 16,
 (18, 107): 16,
 (30, 106): 17,
 (17, 107): 17,
 (29, 106): 18,
 (17, 106): 18,
 (28, 106): 19,
 (16, 106): 19,
 (27, 106): 20,
 (16, 107): 20,
 (26, 106): 21,
 (15, 107): 21,
 (25, 106): 22,
 (14, 107): 22,
 (24, 106): 23,
 (14, 106): 23,
 (23, 106): 24,
 (13, 106): 24,
 (22, 106): 25,
 (12, 106): 25,
 (21, 106): 26,
 (12, 107): 26,
 (20, 106): 27,
 (13, 107): 27,
 (20, 105): 28,
 (13, 108): 28,
 (19, 105): 29,
 (14, 108): 29,
 (19, 104): 30,
 (15, 108): 30,
 (18, 104): 31,
 (16, 108): 31,
 (18, 105):

In [258]:
max(val for val in visited_nodes.values()) 

6701